# Conversational AI - aka Chatbot!

In [1]:
# Importing the Libraries

import requests
import json
import time
import re
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
import gradio as gr

In [3]:
# Initializing System Messages and ollama details

system_message = "You are a helpful assistant"
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME="gemma3"

``chat(message, history)``

Which expects to receive history in a particular format

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```



In [4]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("history is\n", history)
    print("Message is\n", message)

    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    result=""
    try:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    delta = data.get("message", {}).get("content", "")
                    if delta:
                        # Optional: remove unwanted <think> tags or others
                        clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                        result += clean_delta
                        yield result
                except json.JSONDecodeError:
                    continue
    finally:
        # Ensure generator exits cleanly
        yield result

## Gradio Chat UI

In [5]:
gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


history is
 []
Message is
 hello
history is
 [{'role': 'user', 'metadata': None, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello there! How can I help you today? 😊 \n\nDo you have a question, need some information, want to brainstorm, or just want to chat? Let me know!', 'options': None}]
Message is
 what is today's date
history is
 [{'role': 'user', 'metadata': None, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello there! How can I help you today? 😊 \n\nDo you have a question, need some information, want to brainstorm, or just want to chat? Let me know!', 'options': None}, {'role': 'user', 'metadata': None, 'content': "what is today's date", 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Today is Wednesday, November 8th, 2023. \n\nHow’s your day going so far?', 'options': None}]
Message is
 i am arivukkarasu
history is
 [{'role': 'user', 'metadata': None, 'content': 'h

In [6]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."


In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    result=""
    try:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    delta = data.get("message", {}).get("content", "")
                    if delta:
                        # Optional: remove unwanted <think> tags or others
                        clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                        result += clean_delta
                        yield result
                except json.JSONDecodeError:
                    continue
    finally:
        # Ensure generator exits cleanly
        yield result

In [8]:
gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [9]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [10]:
gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [11]:
def chat(message, history):
    final_system_message = system_message
    if "belt" in message:
        final_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": final_system_message}] + history + [{"role": "user", "content": message}]

    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    result=""
    try:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    delta = data.get("message", {}).get("content", "")
                    if delta:
                        # Optional: remove unwanted <think> tags or others
                        clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                        result += clean_delta
                        yield result
                except json.JSONDecodeError:
                    continue
    finally:
        # Ensure generator exits cleanly
        yield result

In [12]:
gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
